# LlamaIndex

LlamaIndex is a framework designed to help you build applications powered by Large Language Models such as chatbots, AI assistants, and translation tools. One of its most valuable capabilities is enriching the knowledge of your LLM with **your own data**, enabling the model to answer questions about **personal, organizational, or domain-specific information** that it wasn’t originally trained on.

> NOTE:
**Before you run this notebook**
- You need an OpenAI API key available as `OPENAI_API_KEY`.
- Cells using LlamaParse require a LlamaCloud API key (`LLAMA_CLOUD_API_KEY`).
- If you do not have these keys, skip the LlamaParse/OpenAI sections and follow the local examples.


# Setup: Installing Required Libraries

Before we begin, we need to install the necessary Python libraries. Run the cell below to install all dependencies for this notebook.

In [1]:
# Install required libraries with working versions
!pip install -q llama-index-core==0.14.6 llama-index-embeddings-openai==0.5.1 \
    llama-index-llms-openai==0.6.6 openai==1.109.1 \
    chromadb==1.2.2 llama-index-vector-stores-chroma==0.5.3 \
    llama-index-readers-file llama-parse

print("✅ All libraries installed successfully!")
print("⚠️  IMPORTANT: Please restart your kernel/runtime now before running the next cell!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 92.0 MB/s eta 0:0

# 1. Data Connectors

LlamaIndex uses data connectors to **ingest information** from a wide range of **structured and unstructured sources**.

The simplest way to load the data is using `SimpleDirectoryReader` which supports various file types such as:

- csv - comma-separated values
- docx - Microsoft Word
- ipynb - Jupyter Notebook
- pdf - Portable Document Format
- ppt, .pptm, .pptx - Microsoft PowerPoint
- ...and many more.

Data connector takes your data from these different formats and put them together in a uniform, organized way so they can be used within your LLM application.

You can find all supported file types in [the documentation](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/#simpledirectoryreader).

Let's import `SimpleDirectoryReader`:

In [1]:
import os

# Configure OpenAI API key
OPENAI_API_KEY = None

try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        print('✅ API key loaded from Colab secrets')
except Exception:
    pass

if not OPENAI_API_KEY:
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    try:
        from getpass import getpass
        print('💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY')
        OPENAI_API_KEY = getpass('Enter your OpenAI API Key: ')
    except Exception as exc:
        raise ValueError('❌ ERROR: No API key provided! Set OPENAI_API_KEY as an environment variable or Colab secret.') from exc

if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == '':
    raise ValueError('❌ ERROR: No API key provided!')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print('✅ Authentication configured!')

OPENAI_MODEL = 'gpt-5-nano'  # Using gpt-5-nano for cost efficiency
print(f'🤖 Selected Model: {OPENAI_MODEL}')

OPENAI_EMBED_MODEL = 'text-embedding-3-small'
print(f'🧠 Embedding Model: {OPENAI_EMBED_MODEL}')


✅ API key loaded from Colab secrets
✅ Authentication configured!
🤖 Selected Model: gpt-5-nano
🧠 Embedding Model: text-embedding-3-small


In [2]:
from llama_index.core import SimpleDirectoryReader

We will load the PDF file called "charter.pdf" (stored in "data" folder in notebook's directory) containing the Charter of Fundamental Rights of the European Union.  

> NOTE: In this notebook we will use the asynchronous (async) versions of data connectors using `await` and `.aload_data()`. It helps everything run more smoothly and prevents technical errors with the notebook’s event loop. You don’t need to understand all the internals, just know that `await` is the keyword that tells Python "this step might take a while, pause here until it’s done".


In [4]:
# Generating documents
documents = await SimpleDirectoryReader(input_files = ["data/charter.pdf"]).aload_data()

When this data connector processes a PDF, it doesn’t treat the whole file as a single block of text. Instead, it splits the PDF into pages and each page is returned as **document object**.

In [5]:
# The number of pages in the original PDF file == The number of document objects
len(documents)

17

Let's display the text of the second document where we can see the Table of Contents:

In [6]:
print(documents[1].text)

 
Table of Contents  
Page 
PREAMBLE . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 393  
TITLE I DIGNITY . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 394  
TITLE II FREEDOMS . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 395  
TITLE III EQUALITY . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 397  
TITLE IV SOLIDARITY . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 399  
TITLE V CITIZENS' RIGHTS . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 401  
TITLE VI JUSTICE . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 403  
TITLE VII GENERAL PROVISIONS GOVERNING THE INTERPRETATION AND 
APPLICATION OF THE CHARTER . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 40

Each document include metadata such as `file_name`, `file_type`, `creation_date`, etc.:

In [7]:
documents[1].metadata

{'page_label': '390',
 'file_name': 'charter.pdf',
 'file_path': 'data/charter.pdf',
 'file_type': 'application/pdf',
 'file_size': 1049657,
 'creation_date': '2025-10-28',
 'last_modified_date': '2025-10-28'}

### 📝 EXERCISE 1: Load and Explore Documents


**Your task:**
1. Check how many document objects were created from the PDF file
2. Display the text content of the first document
3. Print the metadata for the first document
4. Think about: Why is the PDF split into multiple document objects? What does each object represent?

**Hint:**
- Use `len(documents)` to count documents
- Access properties with `documents[0].text` and `documents[0].metadata`



In [8]:
# YOUR CODE HERE


# 2. Creating the Index and Querying
Next, we’ll build a vector database to store our embeddings. We'll use `VectorStoreIndex.from_documents()` which automatically **breaks each document into smaller pieces called nodes** based on length. Each node keeps the metadata of its parent document, so we don’t lose context. Once the nodes are created, they are passed to an embedding model - `text-embedding-ada-002` from OpenAI by default.

In [9]:
# Creating the index
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)

Next, we’ll turn the index into a query engine so that we can ask questions.

Behind the scenes, the workflow looks like this:
1. **Query Embedding**: Our text query is embedded into a vector
2. **Retriever**: Query vector is compared against the embeddings stored in the index and retriever returns the most relevant nodes - LlamaIndex uses **cosine** similarity by default
3. **Response Syntethizer**: Combines the retrieved nodes with our query to generate a prompt, which is then passed to an LLM to produce an answer - LlamaIndex uses `gpt-3.5-turbo` from OpenAI by default.

In [10]:
# Setting the index as query engine
query_engine = index.as_query_engine()

# Querying
print(query_engine.query("What is Title 1 about?"))

Title I is about fundamental human rights and dignity, emphasizing the protection and respect for human life, integrity, and prohibiting practices such as torture, slavery, discrimination, and ensuring equality between men and women.


# 3. Making Data Persistent

By default, `VectorStoreIndex` keeps all data in memory. However, LlamaIndex has its own built-in persistence mechanism.

We will use `persist()` method that handle saving the index into "my_storage". In the code cell below, if folder "my_storage" does not exist yet the code will:
- load PDF file from "data" folder
- build a new index
- persist that index to disk inside "my storage"

If folder "my_storage" already exists, the code instead:
- creates `StorageContext` object pointing to this folder
- reload the previously saved index directly

In [11]:
import os
import os.path
from llama_index.core import StorageContext, load_index_from_storage

# A directory
PERSIST_DIR = "./my_storage"

if not os.path.exists(PERSIST_DIR):
    # Loading the documents and creating the index
    documents = await SimpleDirectoryReader(input_files = ["data/charter.pdf"]).aload_data()
    index = VectorStoreIndex.from_documents(documents)
    # Storing
    index.storage_context.persist(persist_dir = PERSIST_DIR)
else:
    # Reloading the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

Now we can start running queries against it:

In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("Can you summarize Title 2?")
print(response)

Title II focuses on various freedoms. It includes the right to liberty and security, respect for private and family life, protection of personal data, the right to marry and found a family, freedom of thought, conscience, and religion.


### 📝 EXERCISE 2: Query Your Index



**Your task:**
1. Think of a question about the EU Charter document (e.g., "What rights do children have?", "What is Article 10 about?", "What freedoms are protected?")
2. Query the index using your question
3. Print the response
4. Try a second question and compare the answers
5. Think about: How does the answer quality depend on your question phrasing?



In [ ]:
# YOUR CODE HERE


# 4. LlamaParse

If your dataset includes different file types or documents with complex layouts (such as tables, multi-column text or embedded images), you can use `LlamaParse`. This parser is part of LlamaCloud and is designed to convert documents into structured outputs while preserving layout features far more accurately than generic readers.

To use this parser, you’ll first need **LlamaCloud account**. Go to www.llamaindex.ai and sign-up. Then navigate to **API keys** section and click **Generate New Key**. Be sure to copy and store this secret key in a safe place. For security reasons, it will not be shown again in your account.

> NOTE: You can also make your LlamaParse API key and base URL load automatically every time your terminal starts. This way, you don’t have to set them manually in every session. Open your terminal and edit your shell file - type `nano ~/.zshrc`. At the end of the file, add the following lines. Then run `source ~/.zshrc`.
>
> `export LLAMA_CLOUD_API_KEY="YOUR_EU_KEY"`
>
> `export LLAMA_CLOUD_API_BASE="api.cloud.eu.llamaindex.ai"`


In [15]:
import os

# Configure LlamaParse API key
LLAMA_CLOUD_API_KEY = None

try:
    from google.colab import userdata
    LLAMA_CLOUD_API_KEY = userdata.get('LLAMA_CLOUD_API_KEY')
    if LLAMA_CLOUD_API_KEY:
        print('✅ LlamaParse API key loaded from Colab secrets')
except Exception:
    pass

if not LLAMA_CLOUD_API_KEY:
    LLAMA_CLOUD_API_KEY = os.getenv('LLAMA_CLOUD_API_KEY')

if not LLAMA_CLOUD_API_KEY:
    try:
        from getpass import getpass
        print('💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: LLAMA_CLOUD_API_KEY')
        LLAMA_CLOUD_API_KEY = getpass('Enter your LlamaParse API Key: ')
    except Exception as exc:
        raise ValueError(
            '❌ ERROR: No LlamaParse API key provided! Set LLAMA_CLOUD_API_KEY as an environment variable or Colab secret.'
        ) from exc

if not LLAMA_CLOUD_API_KEY or LLAMA_CLOUD_API_KEY.strip() == '':
    raise ValueError('❌ ERROR: No LlamaParse API key provided!')

os.environ['LLAMA_CLOUD_API_KEY'] = LLAMA_CLOUD_API_KEY

print('✅ LlamaParse authentication configured!')


✅ LlamaParse API key loaded from Colab secrets
✅ LlamaParse authentication configured!


In [16]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type = "text",
    base_url = "https://api.cloud.eu.llamaindex.ai",  # Calling the EU LlamaCloud endpoint
    verbose = True
)

In [17]:
documents = await parser.aload_data("./data/charter.pdf")

Started parsing the file under job_id 91e05b9c-4295-4482-bfdd-70bc0e97fb05


Let's again display the text of the second document - the parser preserves layout features like headings better than a simple text extractor like `SimpleDirectoryReader`:

In [18]:
print(documents[1].text)


C 202/390  EN    Official Journal of the European Union    7.6.2016

                 Table of Contents

                                                                                                                                   Page

PREAMBLE      . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .        393

TITLE I       DIGNITY . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .              394

TITLE II      FREEDOMS . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .                 395

TITLE III     EQUALITY              . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .    397

TITLE IV      SOLIDARITY . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .                 399

TITLE V       CITIZENS' RIGHTS                  . . . . . . . . . . . . . . .

## 4.1 Using LlamaParse - PDF with tables into Markdown

Now let’s try `LlamaParse` on PDF called "livestock_poultry.pdf" that contains not only the text but also **several tables**. `LlamaParse` will return the content in **Markdown format** which makes the document far easier for an LLM to interpret.

In the code cell below, we initialize the parser that connects to the LlamaCloud API - we need to set `base_url` that specifies which regional LlamaCloud endpoint to use. In this case, we’re pointing to the EU server.

In [19]:
# Parsing PDF
parser = LlamaParse(
    result_type = "markdown",
    base_url = "https://api.cloud.eu.llamaindex.ai",
    verbose = True
)

Now we can send a PDF file to the parser:

In [20]:
pdf_doc = await parser.aload_data("./data/livestock_poultry.pdf")

Started parsing the file under job_id 65273faa-d7c1-4248-93d5-f2cbc2251196
.

Let's print the document with index 8. Compare this Markdown output with the original PDF (page 9). Notice how the layout is preserved. This is what makes `LlamaParse` valuable: instead of flattening tables into plain text, it captures structure in a way that downstream models can use effectively.

In [21]:
preview = pdf_doc[8].text[:500]
print(preview)



Cattle Stocks - Top Countries Summary

# (in 1,000 head)

# 1. Total Cattle Beg. Stks

| Country        | 2021    | 2022    | 2023    | 2024    | 2025    | 2025    |
| -------------- | ------- | ------- | ------- | ------- | ------- | ------- |
| India          | 305,500 | 306,700 | 307,400 | 307,420 | 307,490 | 307,490 |
| Brazil         | 193,195 | 193,780 | 194,365 | 192,572 | 186,875 | 186,875 |
| China          | 95,621  | 98,172  | 102,160 | 105,090 | 104,000 | 104,900 |
| European Union


### 📝 EXERCISE 3: Compare SimpleDirectoryReader vs LlamaParse



**Your task:**
1. Look at the parsed output from LlamaParse for the livestock_poultry.pdf document
2. Display a different page/document from the parsed results
3. Observe how tables and structured data are represented
4. Think about: When would you use LlamaParse instead of SimpleDirectoryReader?



In [ ]:
# YOUR CODE HERE


## 4.2 Parsing different file types

In this section, we’ll see how to use LlamaParse to handle documents of different types, such as PDFs and Word files, and bring them into a single search workflow.

Instead of writing separate code for each format, we can map file extensions to the same parser and let `SimpleDirectoryReader` automatically process everything in a folder.

First, we'll initialize a parser:

In [22]:
parser = LlamaParse(result_type = "markdown",
                    base_url = "https://api.cloud.eu.llamaindex.ai",
                    verbose = True)

Next, we'll map file extensions to the parser:

In [23]:
file_extractor = {
    ".pdf": parser,
    ".docx": parser
}

Now we can tell `SimpleDirectoryReader` to scan a folder with files "charter.pdf", "livestock_poultry.pdf" and "vacation_policy.docx". If it finds a `.pdf` or `.docx`, it will use our parser to process it. The result is a list of document objects where each page or section is stored as Markdown text.

In [24]:
documents = await SimpleDirectoryReader(
    input_dir = "./data/",
    file_extractor = file_extractor
).aload_data()

Started parsing the file under job_id fbed2a59-b90f-4127-8b0f-a7aa7a231cc4
Started parsing the file under job_id 2a247618-1d15-4b85-b0b1-8fb0fcca6f50
Started parsing the file under job_id 44ec92ec-99ab-4866-916b-8c215900f689


Now we are going to create embeddings for our documents. As we already know, when we build `VectorStoreIndex`, it automatically splits text into chunks before embedding, but this uses default settings.

However, we can use `SentenceSplitter` to gain explicit control over how that chunking happens:
- `chunk_size`: sets the maximum length of each chunk (keeps chunks small enough to fit into the embedding model and LLM context window)
- `chunk_overlap`: defines how much content is repeated between consecutive chunks



**Chunk size matters**

When we embed an entire document as a single vector, we are effectively averaging all of its topics into one point in space. For multi-topic articles this produces a diluted signal: a query about "zero trust" might rank poorly because the vector also carries equally strong signals for other sections such as cryptography or phishing. Chunking breaks the document into focused segments so each embedding represents one coherent idea, dramatically improving retrieval precision.

**Trade-offs**
- *Chunks that are too large (1000+ tokens)* keep the full context but blend unrelated concepts, reducing similarity scores and hurting recall.
- *Chunks that are too small (50 tokens)* deliver crisp matches but may lose the surrounding context the LLM needs when generating an answer.

**Mitigation strategies**
1. Start with a balanced window (e.g., 256–512 tokens) and adjust based on your corpus.
2. Introduce overlap (e.g., 10–20% of the chunk size) so important sentences near boundaries appear in both neighbouring chunks.
3. During retrieval, fetch neighbouring chunks or stitch together the original document spans so the LLM receives enough context to respond reliably.

This approach preserves the semantic focus needed for accurate vector search while still giving the downstream LLM the broader context it needs.



In [25]:
from llama_index.core.node_parser import SentenceSplitter

# Split into nodes (chunks)
splitter = SentenceSplitter(
    chunk_size = 512,        # each chunk will be about 512 characters/tokens long
    chunk_overlap = 50)      # the last 50 characters/tokens of one chunk will also appear at the start of the next

nodes = splitter.get_nodes_from_documents(documents)

The next step is to build a vector index:

In [26]:
# Creating embeddings from "nodes"
index = VectorStoreIndex.from_documents(nodes)

# Wrapping the index in a query engine
query_engine = index.as_query_engine()

In the code cell below, the question is converted into a vector embedding which is compared against all stored embeddings (nodes) in the vector index. The nodes whose embeddings are most similar (highest cosine score) are selected as "relevant" and combined with the query and passed to an LLM to generate the answer:

In [27]:
# Running the query
print(query_engine.query("How many days can be carried over into the next calendar year?"))

There are 15 days that can be carried over into the next calendar year.


In [28]:
# Running the query
print(query_engine.query("What are brazil top five pork export markets?"))

China, Philippines, Chile, Japan, and Hong Kong.


In [29]:
# Running the query
print(query_engine.query("What are the citizens' rights?"))

The citizens' rights include the right to vote and stand as a candidate at elections to the European Parliament, the right to vote and stand as a candidate at municipal elections, the right to good administration, the right of access to documents, the right to refer cases of maladministration to the European Ombudsman, the right to petition the European Parliament, and the freedom of movement and residence within the territory of the Member States.


## 4.5 Using different LLM

Up to now, we’ve built a vector index using the default embedding model and the default LLM. But both of these can be customized. By default, LlamaIndex uses OpenAI’s `text-embedding-ada-002` for embeddings and `gpt-3.5-turbo` for the LLM.

In the example below, we’ll rebuild our index with a different embedding model - `text-embedding-3-small`, and then use a different LLM - `gpt-5-nano` to generate answers:

In [30]:
from llama_index.embeddings.openai import OpenAIEmbedding

# Building a new index + new embedding model
pdf_index = VectorStoreIndex.from_documents(
    pdf_doc,
    embedding = OpenAIEmbedding(model = OPENAI_EMBED_MODEL)
)

In [31]:
from llama_index.llms.openai import OpenAI

# Using new LLM
query_engine = index.as_query_engine(llm = OpenAI(model=OPENAI_MODEL))
response = query_engine.query("What is the forecasted percentage change of global export of pork between 2024 and 2025?")
print(response)

A 1% decrease.
